In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('./model/dinov2-base')
model = AutoModel.from_pretrained('./model/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs[0]

print(outputs)

In [ ]:
hidden_state=last_hidden_states

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LinearClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.fc(x)
        return nn.functional.softmax(x, dim=1)
# Define your training data (hidden state and labels)
# Assuming hidden_state is of shape [batch_size, hidden_size] and labels is of shape [batch_size, num_classes]

# Example:
# hidden_state = torch.randn(batch_size, hidden_size)
# labels = torch.randint(0, num_classes, (batch_size,))

# Define model, loss function, and optimizer
input_size = hidden_state.shape[1]  # assuming hidden state is your input
output_size = 2  # assuming you have this number of classes
model = LinearClassifier(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Set your desired number of epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    optimizer.zero_grad()  # Clear gradients

    # Forward pass
    outputs = model(hidden_state)  # Assuming hidden_state is your input
    loss = criterion(outputs, labels)

    # Backward pass
    loss.backward()  # Calculate gradients
    optimizer.step()  # Update weights

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Optionally, you may want to save the trained model
# torch.save(model.state_dict(), 'linear_classifier.pth')
